A notebook to evaluate Vera C. Rubin Observatory LSST survey strategies for measuring the brown dwarf luminosity function.

Goal: Using the latest brown dwarf luminosity function (BDLF), simulate the numbers of brown dwarfs with good parallaxes. Be able to tell if some strategies are much worse or better than baseline. The unexpected peak in the BDLF at ~1200K may make a specific metric. 

Remarks: Since goal to evaluate survey strategies, should ignore subtleties like spread in colors, unresolved and resolved binaries, etc.  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table

In [2]:
#load the luminosity function as a function of tempeature -- use Kirkpatrick et al. 2020 -- this 
# gives number per cubic parsec.  

#load the empirical absoute magnitudes in i,z,y from PS1.  -- use Best et al. 2018 2018ApJS..234....1B

#we could get empirical spectra instead but I think PS1 magnitudes make more sense.

In [3]:
from astropy.io import ascii

filename='brown_dwarf_data.txt'
bd_data = ascii.read(filename)
# space density units are objects per cubic parsec
# type 1 = L1-L1.5, 10 = T0.0-0.5, etc.  
bd_data

type,spacedensity,Mz,My
int64,float64,float64,float64
0,0.00031,14.52,13.58
1,0.00063,14.93,13.97
2,0.00047,15.3,14.33
3,0.00028,15.88,14.89
4,0.00066,16.68,15.66
5,0.00066,16.94,15.87
6,0.00081,17.35,16.27
7,0.0006,18.18,17.13
8,0.00031,18.1,17.04


We could follow three strategies:

1. Monte Carlo: Place a large number of BDs out to some distance (100pc?) with random positions on sky. Calculate apparent magnitudes for each. parallax uncertainty. 

Question: How do we go from a list of random positions, magnitudes to uncertainty using existing metrics. 

2. Could instead just use one (or more?) reference distance (50 parsecs ?) and then calculate the parallax uncertainty for each spectral type at that distance.  **THIS SEEMS EASIEST.  METRIC MIGHT BE PASS/FAIL, OR CAN MODIFY DISTANCE ONCE WE UNDERSTAND FIRST PASS** 

3. Specify a desired accuracy parallax/sigma = 10 [or 20], calculate the volume probed at each spectral type at each point in the sky. We would get uncertainty from MAF.  **THIS WOULD LEAD TO MOST OBVIOUS METRIC**




POSSIBLE METRICS:

A. Number of BDs measured with sigma/pi>10.0 [at some spectral type??]

B. Volume for T5 dwarfs with sigma/pi>10.0

C. Coolest type with sigma/pi=10 at 50pc

anyway, if we have strategy 1,2, or 3 working we can select an appropriate metric. 

In [4]:
#start strategy 2

ref_distance = 50.0    #pc
ref_parallax= 1000.0/ref_distance  #mas
ref_dm = 5.0 * np.log10(ref_distance)-5.0

In [5]:
bd_data['zmag_ref'] = bd_data['Mz'] + ref_dm
bd_data['ymag_ref'] = bd_data['My'] + ref_dm


In [6]:
bd_data

type,spacedensity,Mz,My,zmag_ref,ymag_ref
int64,float64,float64,float64,float64,float64
0,0.00031,14.52,13.58,18.014850021680093,17.074850021680092
1,0.00063,14.93,13.97,18.424850021680093,17.464850021680093
2,0.00047,15.3,14.33,18.794850021680094,17.824850021680092
3,0.00028,15.88,14.89,19.374850021680096,18.384850021680094
4,0.00066,16.68,15.66,20.174850021680093,19.154850021680094
5,0.00066,16.94,15.87,20.434850021680095,19.36485002168009
6,0.00081,17.35,16.27,20.844850021680095,19.764850021680093
7,0.0006,18.18,17.13,21.674850021680093,20.624850021680093
8,0.00031,18.1,17.04,21.594850021680095,20.534850021680093


next step: Calculate the expected uncertainty in parallax for each of these bins (i.e., each spectral type at the reference distance.) uses only z,y.  can we do this with existing MAF tool?